In [6]:
import requests
import pymysql
import json

In [7]:
ETHSCAN_API_KEY = "66PEZRV8MGWR8VRS6BCNX7VM7BNW3GYXUJ"
ETHSCAN_URL = f"http://api.etherscan.io/api?&apikey={ETHSCAN_API_KEY}&"

In [8]:
host_db = "172.23.96.1"
user_db = "discord-user"
db_name = 'dapp_analysis_rearrange'
password = "vlvlvlvl123"

db = pymysql.connect(host = host_db, user = user_db, db = db_name, password = password)
cursor = db.cursor()

In [10]:
query = "SELECT DISTINCT to_addr FROM tx_extend WHERE to_addr_type = 'contract'"
cursor.execute(query)
contract_addrs = cursor.fetchall()
len(contract_addrs)

124

In [11]:
contract_addrs

(('0x705203fc06027379681aef47c08fe679bc4a58e1',),
 ('0xbad7a99557f80eb6c99993f91778863b35cb564e',),
 ('0x54833d94b55202f2cbba52c3a002131e76fef30b',),
 ('0x9c1065e4a2fe67715ce82772cfc223bd76009451',),
 ('0x653bc9842cb54fa5454645be2eed7265f9fb9523',),
 ('0xdb91670cb86a36067a72a4e73b283bb5cdbd4e58',),
 ('0x9708c86466d084a45ad0569658da30c61912a19f',),
 ('0x94c0d029a7b64bf443e89c5006089364c0d60d61',),
 ('0xa62142888aba8370742be823c1782d17a0389da1',),
 ('0x041b3eb05560ba2670def3cc5eec2aeef8e5d14b',),
 ('0xabb4b9b48ceaefc290464025bdd4ef324f8b23c4',),
 ('0x27f706edde3ad952ef647dd67e24e38cd0803dd6',),
 ('0xf91546835f756da0c10cfa0cda95b15577b84aa7',),
 ('0x43ee79e379e7b78d871100ed696e803e7893b644',),
 ('0x24da016c06941ec2c92be28e0a2b2e679f0d1dc7',),
 ('0x29488e24cfdaa52a0b837217926c0c0853db7962',),
 ('0x417bdc58ef9a3d7de04a66ab84ed13048d9a82bb',),
 ('0x585a546a781176fbb306caa4e4198a0a560964d3',),
 ('0xcb4046e50f71409a3af23da0961b5ce2f769de31',),
 ('0x60be37dacb94748a12208a7ff298f6112365e31f',),


In [9]:
def bytecode(addr):
    resq = requests.get(ETHSCAN_URL + f"module=proxy&action=eth_getCode&address={addr}&tag=latest")
    data = resq.json()
    return data['result']

def contract_detail(addr):
    resq = requests.get(ETHSCAN_URL + f"module=contract&action=getsourcecode&address={addr}")
    data = resq.json()
    return data['result'][0]

In [14]:
def getContract(list_contracts):
    query = "INSERT INTO smart_contract(address,name,bytecode,src_code,detail,isDestroyed,ABI) VALUES (%s,%s,%s,%s,%s,%s,%s)" 
    
    for addr in list_contracts:
        # addr = addr[0]
        
        bytecode_ = bytecode(addr)
        detail = contract_detail(addr)
        isDestroyed = True if bytecode_ == "0x" else False
        extracted_details = {
            'compiler_version' : detail['CompilerVersion'],
            'swarm_code' : detail['SwarmSource'],
            'contructtor_args' : detail['ConstructorArguments'],
            'license_type' : detail['LicenseType'],
            'libs' : detail['Library'],
            
        }
        try:
            abi = json.loads(detail['ABI'])
            abi = json.dumps(abi)
        except:
            abi = None
        
        args = (
            addr, detail['ContractName'],
            bytecode_, detail['SourceCode'],
            json.dumps(extracted_details),
            isDestroyed,
            abi
        )
        cursor.execute(query, args)
        db.commit()
    db.close()
          
getContract(contract_addrs)